#   生产者（多线程） --->   全局变量   --->   消费者（多线程）

### 生产者和消费者模式是多线程开发中经常见到的一种模式。生产者的线程专门用来生产一些数据，然后存放到一个中间的变量中。消费者再从这个中间的变量中取出数据进行消费。但是因为要使用中间变量，中间变量经常是一些全局变量，因此需要使用锁来保证数据完整性。

#### 以下是使用threading.Lock锁实现的“生产者与消费者模式”的一个例子：

In [1]:
import threading
import random
import time

gMoney = 1000
gLock = threading.Lock()
# 记录生产者生产的次数，达到10次就不再生产
gTimes = 0

class Producer(threading.Thread):
    def run(self):
        global gMoney
        global gLock
        global gTimes
        while True:
            money = random.randint(100, 1000)
            gLock.acquire()
            # 如果已经达到10次了，就不再生产了
            if gTimes >= 10:
                gLock.release()
                break
            gMoney += money
            print('%s当前存入%s元钱，剩余%s元钱' % (threading.current_thread(), money, gMoney))
            gTimes += 1
            time.sleep(0.5)
            gLock.release()

class Consumer(threading.Thread):
    def run(self):
        global gMoney
        global gLock
        global gTimes
        while True:
            money = random.randint(100, 500)
            gLock.acquire()
            if gMoney > money:
                gMoney -= money
                print('%s当前取出%s元钱，剩余%s元钱' % (threading.current_thread(), money, gMoney))
                time.sleep(0.5)
            else:
                # 如果钱不够了，有可能是已经超过了次数，这时候就判断一下
                if gTimes >= 10:
                    gLock.release()
                    break
                print("%s当前想取%s元钱，剩余%s元钱，不足！" % (threading.current_thread(),money,gMoney))
            gLock.release()

def main():
    for x in range(5):
        Consumer(name='消费者线程%d'%x).start()

    for x in range(5):
        Producer(name='生产者线程%d'%x).start()

if __name__ == '__main__':
    main()

<Consumer(消费者线程0, started 4692)>当前取出397元钱，剩余603元钱
<Consumer(消费者线程1, started 6672)>当前取出314元钱，剩余289元钱
<Consumer(消费者线程1, started 6672)>当前取出220元钱，剩余69元钱
<Consumer(消费者线程1, started 6672)>当前想取186元钱，剩余69元钱，不足！
<Producer(生产者线程0, started 6728)>当前存入931元钱，剩余1000元钱<Consumer(消费者线程1, started 6672)>当前想取425元钱，剩余69元钱，不足！

<Producer(生产者线程0, started 6728)>当前存入943元钱，剩余1943元钱
<Producer(生产者线程0, started 6728)>当前存入629元钱，剩余2572元钱
<Producer(生产者线程0, started 6728)>当前存入571元钱，剩余3143元钱
<Producer(生产者线程0, started 6728)>当前存入344元钱，剩余3487元钱
<Producer(生产者线程0, started 6728)>当前存入525元钱，剩余4012元钱
<Consumer(消费者线程2, started 6824)>当前取出105元钱，剩余3907元钱
<Consumer(消费者线程2, started 6824)>当前取出144元钱，剩余3763元钱
<Consumer(消费者线程4, started 6584)>当前取出384元钱，剩余3379元钱
<Consumer(消费者线程1, started 6672)>当前取出191元钱，剩余3188元钱
<Consumer(消费者线程1, started 6672)>当前取出433元钱，剩余2755元钱
<Producer(生产者线程2, started 6172)>当前存入864元钱，剩余3619元钱
<Producer(生产者线程2, started 6172)>当前存入913元钱，剩余4532元钱
<Producer(生产者线程4, started 6888)>当前存入680元钱，剩余5212元钱
<Consumer(消费者线程0, started 4692)

#### 下同demo4.py文件：

In [3]:
# Lock版本生产者和消费者模式

import threading
import random
import time

gMoney = 1000
gLock = threading.Lock()
gTotalTimes = 10
gTimes = 0

# 生产者类 (不断生产money)
class Producer(threading.Thread):
    def run(self):
        global gMoney
        global gTimes
        while True:    # 使用while true死循环来不断生产
            money = random.randint(100,1000)
            gLock.acquire()
            if gTimes >= gTotalTimes:
                gLock.release()
                break
            gMoney += money
            print("%s生产了%d元钱，剩余%d元钱" %(threading.current_thread(), money, gMoney))
            gTimes += 1
            gLock.release()
            time.sleep(0.5)

# 消费者类 (断消费money)
class Consumer(threading.Thread):
    def run(self):
        global gMoney
        while True:
            money = random.randint(100,1000)
            gLock.acquire()
            if gMoney >= money:
                gMoney -= money
                print("%s消费了%d元钱,剩余%d元钱" %(threading.current_thread(), money, gMoney))
            else:
                if gTimes >= gTotalTimes:
                    gLock.release()
                    break
                print("%s消费者准备消费%d元钱，剩余%d元钱，余额不足！"%(threading.current_thread(), money, gMoney))
            gLock.release()
            time.sleep(0.5)

def main():
    for x in range(5):
        t = Consumer(name="消费者线程%d"%x)   #指定进程的名字
        t.start()

    for x in range(5):
        t = Producer(name="生产者线程%d"%x)   #指定进程的名字
        t.start()

if __name__ == '__main__':
    main()

<Consumer(消费者线程0, started 6016)>消费了757元钱,剩余243元钱
<Consumer(消费者线程2, started 6160)>消费者准备消费505元钱，剩余243元钱，余额不足！<Consumer(消费者线程1, started 4188)>消费者准备消费811元钱，剩余243元钱，余额不足！<Consumer(消费者线程3, started 6580)>消费者准备消费802元钱，剩余243元钱，余额不足！


<Consumer(消费者线程4, started 5796)>消费者准备消费700元钱，剩余243元钱，余额不足！<Producer(生产者线程0, started 6692)>生产了176元钱，剩余419元钱<Producer(生产者线程1, started 5568)>生产了828元钱，剩余1247元钱


<Producer(生产者线程2, started 6860)>生产了611元钱，剩余1858元钱
<Producer(生产者线程3, started 6704)>生产了839元钱，剩余2697元钱
<Producer(生产者线程4, started 3060)>生产了369元钱，剩余3066元钱
<Consumer(消费者线程0, started 6016)>消费了363元钱,剩余2703元钱
<Consumer(消费者线程1, started 4188)>消费了584元钱,剩余2119元钱
<Consumer(消费者线程2, started 6160)>消费了301元钱,剩余1818元钱
<Consumer(消费者线程3, started 6580)>消费了406元钱,剩余1412元钱
<Consumer(消费者线程4, started 5796)>消费了375元钱,剩余1037元钱
<Producer(生产者线程0, started 6692)>生产了621元钱，剩余1658元钱
<Producer(生产者线程1, started 5568)>生产了282元钱，剩余1940元钱
<Producer(生产者线程2, started 6860)>生产了561元钱，剩余2501元钱
<Producer(生产者线程3, started 6704)>生产了271元钱，剩余2772元钱
<Producer(生产者线程4